# 🧠 AI Agents

This notebook demonstrates how to build a **LangChain agent** with multiple tools.

---

## 🔍 What is an Agent?

An **agent** is like an intelligent assistant that:
1. Thinks about the user's request (**Thought**)
2. Decides what tool(s) to use (**Action**)
3. Executes the tool(s) and observes results (**Observation**)
4. Returns the final response (**Final Answer**)

This is based on the **ReAct framework** (Reason + Act), which gives LLMs structured reasoning steps.

This means that ReAct agents can:
- **Think** about the task
- **Act** by calling tools
- **Observe** the results of their actions

---

## 🔧 What are Tools?

- Tools are **functions** (APIs, calculators, web search, etc.) that the agent can call.
- Each tool has:
  - A **name**
  - A **description** (so the agent knows when to use it)
  - A **function** that does the actual work


In [ ]:
from dotenv import load_dotenv
import warnings
from langchain_groq import ChatGroq

In [ ]:
# load environment variables from .env file
load_dotenv()

### Different Models which can be used

[Here](https://console.groq.com/docs/deprecations) you will find all the models available other than which are listed below.

| Model                   | Size / Type        | Context Window | Strengths                                                                 | Best Use Cases                                      |
|--------------------------|-------------------|----------------|---------------------------------------------------------------------------|----------------------------------------------------|
| **gemma2-9b-it**         | 9B, instruction-tuned | 8K             | Google’s Gemma v2 instruction-tuned model. Good at following prompts, lightweight compared to 70B models. | Small-to-mid agents, lightweight reasoning, fine for simple tool use with prompting. |
| **llama-3.3-70b-versatile** | 70B, general versatile | 8K             | Meta’s flagship Llama 3.3 model hosted on Groq. High reasoning quality, supports structured outputs and function calling. | Complex agents, production-grade apps, when you need reliability and accuracy. |
| **llama-3.1-8b-instant** (not working in this example)   | 8B, fast & compact | 128K           | Optimized for **speed** with large 128K context window. Supports JSON mode and function calling, very low latency. | Real-time apps, chatbots, smaller agents where fast responses matter. |


In [ ]:
# Get llms models via the `langchain_groq` package
llm_model = ChatGroq(
    model="llama-3.3-70b-versatile", #llama-3.3-70b-versatile",
    temperature=0,
    max_tokens=512,
)

### Defining Tools 🔨
We want to equip our agent with 2 simple custome-made tools:
1. **MathTool**: Solves math expressions using Python's `eval()`.
2. **LengthTool**: Counts the number of characters in a string.

In [ ]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType


##### Custome-made Tool 1

In [ ]:
# Tool 1
def math_calculator(input_string: str) -> str:
    """A simple calculator tool that performs arithmetic operations."""
    # Evaluate the input string as a mathematical expression    
    result = eval(input_string)
    return f"The result is: {result}"
    

In [ ]:
math_tool = Tool(
    name="MathTool", # Name of the tool
    func=math_calculator, # Function to call
    description="Use this tool to solve basic math problems. Input should be a valid math expression like '8 * 12'." # Description of the tool
)

##### Custom Tool 2

In [ ]:
# Tool 2
def string_length(input_string: str) -> str:
    """A tool to count the number of characters in a string."""
    length = len(input_string)
    return f"The length of the string is: {length} characters."

In [ ]:
length_tool = Tool(
    name="LengthTool",
    func=string_length,
    description="Use this tool to count the number of characters in a string."
)

> __Very important__:
> 
>  Add __clear descriptions__ for each tools 🛠️!!!
> 
> The agent **doesn't know how the tools work internally**.  
> 
> It relies entirely on the tool's **name** and **description** to decide when to use them.

### Initialize the Agent
We create a type of agent called `zero-shot-react-description`, which means:
- It uses the **ReAct** reasoning framework:
  - **Thought**: Decide what to do
  - **Action**: Pick a tool
  - **Observation**: See the tool's output
  - Repeat until it has the **Final Answer**
- **Zero-shot** means the agent doesn't need any examples.  
  It figures out how to use tools **just from their descriptions**.

##### 🧠 LangChain Agent Types — Overview

| AgentType | Description | Best For / Use Case |
|-----------|-------------|---------------------|
| `AgentType.ZERO_SHOT_REACT_DESCRIPTION` | Classic ReAct-style agent. Chooses tools by reasoning step-by-step from scratch using only descriptions. | Simple tool use, when you want the agent to figure out how to solve tasks on its own. |
| `AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION` | Same as above, but optimized for chat models (e.g., gpt-3.5-turbo). Uses `ChatPromptTemplate`. | Chat-based apps with tool use. |
| `AgentType.REACT_DOCSTORE` | Uses ReAct logic, but built for interacting with a single knowledge base (like Wikipedia). | Agents that need to read docs or search a single data source. |
| `AgentType.SELF_ASK_WITH_SEARCH` | Specialized agent for answering factual questions. Agent asks itself sub-questions, then answers them using a search tool. | Fact-based Q&A (like "What year did X happen?"). Often paired with web search. |
| `AgentType.CONVERSATIONAL_REACT_DESCRIPTION` | Like `ZERO_SHOT_REACT_DESCRIPTION`, but keeps track of conversation history (with memory). | Conversational agents using tools. Good for chatbots. |
| `AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION` | Chat version of the above. Uses `ChatPromptTemplate` with memory and tool reasoning. | Best for multi-turn chats + tools. |
| `AgentType.OPENAI_FUNCTIONS` | Uses OpenAI's function calling (like GPT-4 tools API). LLM chooses tool via function schema. | If you're using OpenAI and want tools selected via native function-calling interface. |
| `AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION` | Newer ReAct-like chat agent that formats output as JSON (structured actions). Works well with function schemas. | When you want tool calls to be structured and parsable (e.g., APIs, apps). |
| `AgentType.OPENAI_MULTI_FUNCTIONS` | Same as `OPENAI_FUNCTIONS`, but allows calling **multiple tools at once**. | Advanced OpenAI workflows needing multi-function execution. |


In [ ]:
dir(AgentType)[:9]

In [ ]:
langchain_agent = initialize_agent(
    tools=[math_tool, length_tool], # List of tools to use
    llm=llm_model, # The language model to use
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # Type of agent to create
    verbose=True, # Whether to print out the agent's reasoning
    max_iterations=3, # Maximum number of iterations for the agent
    return_intermediate_steps=False, # Whether to return intermediate steps
)

### Run the Agent 🤖

In [ ]:
# Example 1: Math question
print("---- Math Question ----")
response = langchain_agent.invoke("What is 15 * 4?")
print("✅ Final Answer:", response)

The output shows the agent's chain of reasoning:
+ tought: reasoning steps
+ action: which tool it chose
+ observation: the result of the tool call

In [ ]:
# Example 2: String length question
print("\n---- String Length Question ----")
response = langchain_agent.invoke("How many characters are in the string '' ?")
print("✅ Final Answer:", response['output'])


## Summary
- Agents are intelligent assistants that can think, act, and observe.
- They use tools to perform tasks.
- LangChain provides a framework for building agents with structured reasoning.
- You can create custom tools and define how the agent should use them.